In [1]:
import numpy as np

#ポケモン一体の状態ベクトル化
def encode_pokemon(pokemon):
    #pokemonは辞書型で受け取る
    species_dict = {'ピカチュウ': 0, 'リザードン': 1, 'カビゴン': 2} #種族ID割り振り

    species_id = species_dict.get(pokemon['species'], -1)
    hp_ratio = pokemon['hp_ratio']
    status_map = {'なし': 0, 'まひ': 1, 'やけど': 2}
    status_id = status_map.get(pokemon['status'], 0)
    pp = np.array(pokemon['pp']) / 15  # PP最大15として正規化

    # ベクトル結合
    vector = np.array([species_id, hp_ratio, status_id])
    vector = np.concatenate([vector, pp])
    return vector

# テスト
poke_example = {'species': 'ピカチュウ', 'hp_ratio': 0.8, 'status': 'なし', 'pp': [5, 10, 10, 15]}
print(encode_pokemon(poke_example))

[0.         0.8        0.         0.33333333 0.66666667 0.66666667
 1.        ]


In [2]:
import numpy as np

def encode_pokemon(pokemon, species_dict, type_dict, ability_dict, item_dict):
    #種族(one-hot)
    species_vector = np.zeros(len(species_dict))
    if pokemon['species'] in species_dict:
        species_vector[species_dict[pokemon['species']]] = 1

    #HP
    hp_vector = np.array([pokemon['hp_ratio']]) #0.0~1.0

    #状態異常
    status_map = {'none': 0, 'par': 1, 'brn': 2, 'slp': 3, 'frz': 4, 'psn': 5, 'tox': 6}
    status_vector = np.zeros(len(status_map))
    if pokemon['status'] in status_map:
        status_vector[status_map[pokemon['status']]] = 1
    sleep_turns = np.array([pokemon.get('sleep_turns', 0) / 3])  # 最大3ターン仮定

    #技情報
    moves_vector = []
    for move in pokemon['moves']:
        type_id = type_dict.get(move['type'], 0)
        move_vec = [
            type_id / len(type_dict),       # タイプID正規化
            move['power'] / 250 if move['power'] else 0,  # 威力（最大250想定）
            move['priority'],               # 優先度（-7〜+5）
            move['pp'] / move['max_pp'],    # 残PP割合
            move['accuracy'] / 100 if move['accuracy'] else 1  # 命中率
        ]
        moves_vector.extend(move_vec)
    moves_vector = np.array(moves_vector)

    # 能力ランク（-6〜+6 → 0〜1に正規化） 
    stat_stages = np.array([(s + 6) / 12 for s in pokemon['stat_stages']])

    # 特性（one-hot）
    ability_vector = np.zeros(len(ability_dict))
    if pokemon['ability'] in ability_dict:
        ability_vector[ability_dict[pokemon['ability']]] = 1

    # 持ち物（one-hot)
    item_vector = np.zeros(len(item_dict))
    if pokemon['item'] in item_dict:
        item_vector[item_dict[pokemon['item']]] = 1

    # 結合
    return np.concatenate([
        species_vector,
        hp_vector,
        status_vector,
        sleep_turns,
        moves_vector,
        stat_stages,
        ability_vector,
        item_vector
    ])